## Función describe_df

In [ ]:
from scipy import stats
import pandas as pd

def describe_df(df):
    '''
    Descripción: Para cada columna del dataframe, devuelve el tipo de variable, el
    porcentaje de nulos, los valores únicos y porcentaje de cardinalidad.

    Argumentos:
    df (dataframe)

    Retorna:
    dataframe: dataframe con las columnas del dataframe original, y con las filas
    de los tipos de variables, el tanto por ciento de nulos, los valores únicos
    y el porcentaje de cardinalidad.
    '''

    # Data_type
    data_type = df.dtypes

    # Nulos
    nulos = df.isna().sum()
    porcentaje_nulos = (nulos / len(df) * 100).round(2)

    # Valores únicos
    unicos = df.nunique()

    # Cardinalidad
    card = (unicos / len(df) * 100).round(2)

    # Creación tabla
    tabla = pd.DataFrame({
        "DATA_TYPE": data_type,
        "MISSINGS (%)": porcentaje_nulos,
        "UNIQUE_VALUES": unicos,
        "CARDIN (%)": card
    },
    index = pd.Index(df.columns, name = "COL_N"))

    return tabla.T # devuelve la traspuesta
 

## Función tipifica_variables

In [ ]:
def tipifica_variables(df, umbral_categoria, umbral_continua):
    '''
    Descripción:
    Clasifica las variables del dataframe df en Binaria, Categórica, Numérica continua
    o Numérica Discreta, según el siguiente criterio:
    - Si la cardinalidad es 2, asignara "Binaria"
    - Si la cardinalidad es menor que `umbral_categoria` asignara "Categórica"
    - Si la cardinalidad es mayor o igual que `umbral_categoria`, entonces entra en juego el tercer argumento:
        * Si además el porcentaje de cardinalidad es superior o igual a `umbral_continua`, asigna "Numerica Continua"
        * En caso contrario, asigna "Numerica Discreta"

    Argumentos:
    df (dataframe)
    umbral_categoria (entero): para determinar si una variable es categórica
    umbral_continua (float): para determinar si una variable numérica es continua o discreta

    Retorna:
    dataframe: dataframe con dos columnas "nombre_variable" y "tipo_sugerido".
    '''

    # 

    df_descripcion = describe_df(df).T # aprovechamos la función describe_df que ya calcula la cardinalidad y hacemos la transpuesta, para que los nombres de
                                     # las variables sean las filas y no las columnas

    # Corrección para cuando solo tengo un valor
    df_descripcion.loc[df_descripcion["UNIQUE_VALUES"] == 1, "CARDIN (%)"] = 0.00

    # Creamos una nueva columna "tipo_sugerido", partiendo de que todas son categóricas por ejemplo
    df_descripcion["tipo_sugerido"] = "Categórica"

    # Clasificamos
    df_descripcion.loc[df_descripcion["UNIQUE_VALUES"] == 2, "tipo_sugerido"] = "Binaria"
    df_descripcion.loc[df_descripcion["UNIQUE_VALUES"] >= umbral_categoria, "tipo_sugerido"] = "Numérica discreta"
    df_descripcion.loc[df_descripcion["CARDIN (%)"] >= umbral_continua, "tipo_sugerido"] = "Numérica continua"

    # Reseteamos el index para que pase a ser la columna "nombre_variable"
    df_descripcion = df_descripcion.reset_index(names = ["nombre_variable"])

    # Eliminamos columnas dejando solo "nombre_variable" y "tipo_sugerido"
    df_descripcion = df_descripcion.drop(columns = ["DATA_TYPE", "MISSINGS (%)", "UNIQUE_VALUES", "CARDIN (%)"])

    return df_descripcion

# Función get_features_num_regression

In [ ]:

from scipy import stats
import pandas as pd
def get_features_num_regression(df, target_col, umbral_corr, pvalue = None):
    '''
    Descripción:
    Selecciona columnas numéricas que tengan una relación fuerte con la variable objetivo. 
    Solo se eligen las que superan el umbral de correlación y opcionalmente las que tienen un p-valor significativo.

    Argumentos: 
    df (dataframe)
    target_col (columna): columna de df, que debería ser el target de un hipotético modelo de regresión. Es una variable numérica con alta cardinalidad.
    umbral_corr (float): debe estar entre 0 y 1.
    pvalue (float, por defecto None): 

    Retorna:
    Lista con los nombres de las columnas numéricas seleccionadas que cumplen los criterios de correlación y, si aplica, de p-valor.
    En caso de error en los parámetros, retorna None.

    '''
    # 1. Comprobaciones básicas

    # Comprobamos que target_col exista en el dataframe  
    if target_col not in df.columns:
        print(f"ERROR: la columna '{target_col}' no existe en el DataFrame.")
        return None

    # Comprobamos que target_col es numérica
    if df[target_col].dtype not in ["int64", "float64"]:
        print("ERROR: 'target_col' debe ser una columna numérica.")
        return None
    
    # Comprobamos que umbral_corr está entre 0 y 1
    if not isinstance(umbral_corr, (int, float)) or not (0 <= umbral_corr <= 1):
        print("ERROR: 'umbral_corr' debe ser un número entre 0 y 1.")
        return None
    
    # Comprobamos pvalue si no es None
    if pvalue is not None:
        if not isinstance(pvalue, (int, float)) or not (0 < pvalue < 1):
            print("ERROR: 'pvalue' debe ser un número entre 0 y 1 o None.")
            return None

     # 2. Seleccionar columnas numéricas 
    
    columnas_numericas = []

    for col in df.columns:
        # añadimos solo columnas numéricas y que no sean el target
        if df[col].dtype in ["int64", "float64"] and col != target_col:
            columnas_numericas.append(col)

     # Si no hay columnas numéricas, devolvemos lista vacía
    if len(columnas_numericas) == 0:
        print("No hay columnas numéricas aparte del target.")
        return []

    columnas_seleccionadas = []
        
    # 3. Calcular correlaciones y p-valores
   
    for col in columnas_numericas:

        # Seleccionamos target + columna actual y quitamos filas con nulos
        datos = df[[target_col, col]].dropna()

        # evitar columnas con muy pocos datos
        if len(datos) < 3:
            continue
        # calculamos la correlación entre el target y una variable numérica,
        try:    
            # Cálculo de correlación y p-valor
            # stats.pearsonr(x, y) devuelve:
            #   - correlación entre x e y
            #   - p-valor del test de hipótesis
            corr, p_val = stats.pearsonr(datos[target_col], datos[col])
            
        except:
            continue

    # 4. Filtro por correlación
     
        if abs(corr) <= umbral_corr:
            continue

    # 5. Filtro por p-valor

        if pvalue is not None:
            if p_val > pvalue:
                continue

        columnas_seleccionadas.append(col)


    return columnas_seleccionadas

# Función plot_features_num_regression

In [1]:

def plot_features_num_regression(df, target_col, columns, umbral_corr, pvalue):
    '''
    Descripción: 
    Selecciona las columnas numéricas del dataframe cuya relación con la variable objetivo supera un umbral indicado. Opcionalmente, 
    también filtra por significación estadística si se proporciona un valor p.

    Argumentos:
        df (DataFrame): 
            Conjunto de datos que contiene la variable objetivo y las 
            variables numéricas a analizar.
        target_col (str): 
            Nombre de la columna objetivo. Debe ser numérica continua 
            o discreta con alta cardinalidad.
        umbral_corr (float): 
            Valor entre 0 y 1 que define la relación mínima exigida para 
            considerar una variable relevante.
        pvalue (float o None): 
            Umbral de significación estadística. Si es None, no se aplica 
            este filtro adicional.

    Retorna:
        list: 
            Lista de nombres de columnas numéricas que cumplen los criterios 
            establecidos. Devuelve None si los argumentos no son válidos.
    '''
            

     #  COMPROBACIONES BÁSICAS
   
    # Comprobamos el target en el DataFrame
    if target_col not in df.columns:
        print("ERROR: la columna objetivo no existe.")
        return None

    # Comprobamos si el El target es numérico
    if df[target_col].dtype not in ["int64", "float64"]:
        print("ERROR: el target debe ser numérico.")
        return None

    # Comprobamos el umbral
    if not (0 <= umbral_corr <= 1):
        print("ERROR: umbral_corr debe estar entre 0 y 1.")
        return None

    # Comprobamos si el pvalue es válido
    if pvalue is not None:
        if not (0 < pvalue < 1):
            print("ERROR: pvalue debe ser un número entre 0 y 1 o None.")
            return None

    #2. Usaremos la columnas numéricas
    if columns == []:
        
        columnas_numericas = [
            col for col in df.columns
            if df[col].dtype in ["int64", "float64"] and col != target_col
        ]
    else:
        
        columnas_numericas = [
            col for col in columns
            if col in df.columns and df[col].dtype in ["int64", "float64"]
        ]

    # Si no hay columnas numéricas:
    if len(columnas_numericas) == 0:
        print("No hay columnas numéricas válidas para representar.")
        return []

    # 3. Usamos get_features_num_regression 
    # Creamos un DF reducido solo con target + columnas numéricas
    df_reducido = df[[target_col] + columnas_numericas]

    # Seleccionar variables relevantes
    seleccionadas = get_features_num_regression(
        df_reducido,
        target_col,
        umbral_corr,
        pvalue
    )

    # Si no hay variables seleccionadas:
    if seleccionadas is None:
        return None

    if len(seleccionadas) == 0:
        print("Ninguna variable numérica supera el umbral de correlación.")
        return []

    # 4. Realizamos los gráficos
       
    max_cols_plot = 5

    for i in range(0, len(seleccionadas), max_cols_plot - 1):       
        batch = seleccionadas[i:i + (max_cols_plot - 1)]

        cols_plot = [target_col] + batch

        # Creamos el pairplot
        sns.pairplot(df[cols_plot].dropna())
        plt.show()

  
    return seleccionadas


# Función get_features_cat_regression

In [ ]:
def get_features_cat_regression(df, target_col, pvalue = 0.05):
    '''
    Descripción:

    Argumentos:
    df (dataframe)
    target_col (columna):
    pvalue (float):

    Retorna:
    
    '''

# Función plot_features_cat_regression

In [ ]:
def plot_features_cat_regression(df, target_col = "", columns = [], pvalue = 0.05, with_individual_plot = False):
    '''
    Descripción:

    Argumentos:
    df(dataframe)
    target_col (columna):
    columns (lista, por defecto vacía):
    pvalue (float, por defecto None):
    with_individual_plot (booleano):

    Retorna:
    
    '''